<a href="https://colab.research.google.com/github/olumideadekunle/Conversational-AI-Chatbot/blob/main/Conversational_AI_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Natural Language Processing
## DeepTech

# Olumide Adekunle

# Hands-On: Building a Simple Chatbot

### Step 1: Import Libraries

In [ ]:
!pip install transformers
import random
from transformers import pipeline

### Step 2: Simple Rule-Based Chatbot

In [ ]:
responses = {
    "greeting": ["Hello!", "Hi there!", "Welcome!"],
    "booking": ["Sure! What date would you prefer?", "When would you like to book?"],
    "farewell": ["Goodbye!", "Have a great day!"]
}

def chatbot_response(user_input):
    if "hello" in user_input.lower():
        return random.choice(responses["greeting"])
    elif "book" in user_input.lower():
        return random.choice(responses["booking"])
    elif "bye" in user_input.lower():
        return random.choice(responses["farewell"])
    else:
        return "I'm not sure I understand that."

# Test the chatbot
user_input = "Hello, I want to book an appointment."
print("Chatbot:", chatbot_response(user_input))

# Hands-On with Hugging Face for Response Generation

### Step 1: Install and Import Libraries

In [ ]:
#!pip install transformers
from transformers import pipeline

### Step 2: Load a Pretrained Model for Chatbots
Use a text-generation pipeline with GPT-2.

In [ ]:
chatbot = pipeline("text-generation", model="gpt2")

### Step 3: Generate a Response

In [ ]:
user_input = "What is artificial intelligence?"
response = chatbot(user_input, max_length=50, num_return_sequences=1)
print("Chatbot Response:", response[0]['generated_text'])

# Hands-On Chatbot Development with Transformers

#### Objective

Build a chatbot using a pretrained transformer model for response generation.

### Step 1: Load Pretrained Model and Tokenizer

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "microsoft/DialoGPT-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

### Step 2: Tokenize User Input and Generate Response

In [ ]:
user_input = "Hello! How are you?"
input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors="pt")
response_ids = model.generate(input_ids, max_length=50, pad_token_id=tokenizer.eos_token_id)
response = tokenizer.decode(response_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
print(f"Chatbot: {response}")

# Task
Enhance the existing chatbot by implementing intent recognition and entity extraction, managing conversation context, evaluating its performance with relevant metrics, developing a simple interactive user interface, and creating comprehensive project documentation including a README, project name, description, presentation outline, and an architectural diagram. Finally, review all components to ensure a well-rounded and professionally presented project.

## Implement Intent Recognition and Entity Extraction

### Subtask:
Enhance the chatbot's understanding by adding a layer for identifying user intent (e.g., 'booking', 'greeting') and extracting key entities (e.g., 'date', 'product'). This can be done using libraries like SpaCy or by training a small classifier.


**Reasoning**:
The first instruction is to expand the existing `responses` dictionary and create a new `recognize_intent` function. I will update the `responses` dictionary with more intents and then define the `recognize_intent` function to identify these intents based on user input keywords. This will enhance the chatbot's ability to classify user queries.



In [ ]:
responses = {
    "greeting": ["Hello!", "Hi there!", "Welcome!"],
    "booking": ["Sure! What date would you prefer?", "When would you like to book?"],
    "farewell": ["Goodbye!", "Have a great day!"],
    "product_inquiry": ["Which product are you interested in?", "Tell me more about the product you're looking for."],
    "support": ["How can I assist you with support?", "Please describe your issue."],
    "unknown": ["I'm not sure I understand that."]
}

def recognize_intent(user_input):
    user_input_lower = user_input.lower()
    if "hello" in user_input_lower or "hi" in user_input_lower:
        return "greeting"
    elif "book" in user_input_lower or "appointment" in user_input_lower:
        return "booking"
    elif "bye" in user_input_lower or "goodbye" in user_input_lower:
        return "farewell"
    elif "product" in user_input_lower or "item" in user_input_lower:
        return "product_inquiry"
    elif "support" in user_input_lower or "help" in user_input_lower or "issue" in user_input_lower:
        return "support"
    else:
        return "unknown"

# Test the intent recognition
print("Intent for 'Hello, I want to book an appointment.':", recognize_intent("Hello, I want to book an appointment."))
print("Intent for 'I need help with my product.':", recognize_intent("I need help with my product."))
print("Intent for 'Tell me about an item.':", recognize_intent("Tell me about an item."))
print("Intent for 'See you later!':", recognize_intent("See you later!"))
print("Intent for 'What is your name?':", recognize_intent("What is your name?"))

**Reasoning**:
Following the subtask instructions, the next step is to install the SpaCy library and download the `en_core_web_sm` English language model. This is necessary for entity extraction in later steps.



In [ ]:
get_ipython().system('pip install spacy && python -m spacy download en_core_web_sm')

**Reasoning**:
The next step is to import the SpaCy library, load the `en_core_web_sm` model, and define the `extract_entities` function as per the subtask instructions. This function will use SpaCy to identify and return named entities from user input.



In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

def extract_entities(user_input):
    doc = nlp(user_input)
    entities = {}
    for ent in doc.ents:
        if ent.label_ not in entities:
            entities[ent.label_] = []
        entities[ent.label_].append(ent.text)
    return entities

# Test the entity extraction
print("Entities for 'I want to book a meeting for tomorrow at 3 PM about the new product launch.':", extract_entities("I want to book a meeting for tomorrow at 3 PM about the new product launch."))
print("Entities for 'Can I get support for my iPhone on July 20th?':", extract_entities("Can I get support for my iPhone on July 20th?"))
print("Entities for 'I am looking for a new laptop.':", extract_entities("I am looking for a new laptop."))

**Reasoning**:
The next step is to modify the `chatbot_response` function to incorporate both `recognize_intent` and `extract_entities` for generating more intelligent responses, as per the subtask instructions.



In [ ]:
def chatbot_response(user_input):
    intent = recognize_intent(user_input)
    entities = extract_entities(user_input)

    response_text = random.choice(responses.get(intent, responses["unknown"]))

    if intent == "booking" and "DATE" in entities:
        date = entities["DATE"][0]
        response_text += f" How about {date}?"
    elif intent == "product_inquiry" and ("PRODUCT" in entities or "NORP" in entities):
        product = entities.get("PRODUCT", entities.get("NORP", [""]))[0]
        response_text += f" Can you tell me more about the {product}?"

    return response_text

# Test the enhanced chatbot
print("Chatbot (greeting + booking entities):", chatbot_response("Hello, I'd like to book an appointment for tomorrow."))
print("Chatbot (product inquiry with product entity):", chatbot_response("I'm looking for an iPhone."))
print("Chatbot (simple farewell):", chatbot_response("Goodbye."))
print("Chatbot (unknown intent):", chatbot_response("What is the weather like?"))


## Implement Conversation Context Management

### Subtask:
Modify the chatbot to maintain conversation history across multiple turns, allowing it to respond more coherently and remember previous interactions.


**Reasoning**:
To implement conversation context management, I need to define `conversation_history`, modify the `chatbot_response` function to accept and update this history, and add logic to use the history for more coherent responses, especially for the 'booking' intent where a date might be missing from the current input.



In [ ]:
conversation_history = []

def chatbot_response(user_input, history):
    intent = recognize_intent(user_input)
    entities = extract_entities(user_input)

    # Append current user input to history
    history.append({"role": "user", "text": user_input})

    response_text = random.choice(responses.get(intent, responses["unknown"]))

    # Enhanced logic for booking intent
    if intent == "booking":
        date_found = False
        if "DATE" in entities:
            date = entities["DATE"][0]
            response_text = random.choice(responses["booking"]) + f" How about {date}?"
            date_found = True
        else:
            # Check conversation history for a date if not found in current input
            for entry in reversed(history):
                if entry["role"] == "user":
                    prev_entities = extract_entities(entry["text"])
                    if "DATE" in prev_entities:
                        date = prev_entities["DATE"][0]
                        response_text = random.choice(responses["booking"]) + f" Based on our previous chat, how about {date}?"
                        date_found = True
                        break
            if not date_found:
                response_text = random.choice(responses["booking"]) + " What date would you prefer?"
    elif intent == "product_inquiry" and ("PRODUCT" in entities or "NORP" in entities):
        product = entities.get("PRODUCT", entities.get("NORP", [""]))[0]
        if product: # Ensure product is not empty
             response_text = random.choice(responses["product_inquiry"]) + f" Can you tell me more about the {product}?"
        else:
             response_text = random.choice(responses["product_inquiry"])
    else:
        response_text = random.choice(responses.get(intent, responses["unknown"]))

    # Append chatbot response to history
    history.append({"role": "chatbot", "text": response_text})

    return response_text

# Test the enhanced chatbot with conversation history
conversation_history = []

print("User: Hello, I'd like to book an appointment.")
resp = chatbot_response("Hello, I'd like to book an appointment.", conversation_history)
print("Chatbot:", resp)

print("User: I want it for next Tuesday.")
resp = chatbot_response("I want it for next Tuesday.", conversation_history)
print("Chatbot:", resp)

print("User: I'm looking for an iPhone.")
resp = chatbot_response("I'm looking for an iPhone.", conversation_history)
print("Chatbot:", resp)

print("User: Goodbye.")
resp = chatbot_response("Goodbye.", conversation_history)
print("Chatbot:", resp)

print("User: What is the weather like?")
resp = chatbot_response("What is the weather like?", conversation_history)
print("Chatbot:", resp)

print("Full Conversation History:", conversation_history)

**Reasoning**:
The previous code block for context management did not fully achieve the desired behavior, particularly for follow-up questions related to booking dates and product inquiries. The `recognize_intent` function's order of checks caused specific intents like 'booking' to be overlooked if 'greeting' keywords were also present. Additionally, the `chatbot_response` function's logic needed to more explicitly leverage the previous conversation turn's intent when the current intent is 'unknown' but contains relevant entities. This refined code addresses these issues by reordering intent recognition, adding specific product keywords to intent recognition, and enhancing the `chatbot_response` to consider historical intent for better context handling.



In [ ]:
responses = {
    "greeting": ["Hello!", "Hi there!", "Welcome!"],
    "booking": ["Sure! What date would you prefer?", "When would you like to book?"],
    "farewell": ["Goodbye!", "Have a great day!"],
    "product_inquiry": ["Which product are you interested in?", "Tell me more about the product you're looking for."],
    "support": ["How can I assist you with support?", "Please describe your issue."],
    "unknown": ["I'm not sure I understand that."]
}

def recognize_intent(user_input):
    user_input_lower = user_input.lower()
    # Prioritize more specific intents first
    if "book" in user_input_lower or "appointment" in user_input_lower:
        return "booking"
    elif "product" in user_input_lower or "item" in user_input_lower or "iphone" in user_input_lower or "laptop" in user_input_lower:
        return "product_inquiry"
    elif "support" in user_input_lower or "help" in user_input_lower or "issue" in user_input_lower:
        return "support"
    elif "hello" in user_input_lower or "hi" in user_input_lower:
        return "greeting"
    elif "bye" in user_input_lower or "goodbye" in user_input_lower:
        return "farewell"
    else:
        return "unknown"

def chatbot_response(user_input, history):
    current_intent = recognize_intent(user_input)
    current_entities = extract_entities(user_input)

    # Determine effective intent, considering history
    effective_intent = current_intent
    if current_intent == "unknown" and history:
        last_user_entry = None
        for entry in reversed(history): # Iterate backwards to find the last user input
            if entry["role"] == "user":
                last_user_entry = entry
                break
        if last_user_entry:
            last_user_intent = recognize_intent(last_user_entry["text"])
            # If the last user input had a booking intent and current input has a DATE entity, assume booking
            if last_user_intent == "booking" and "DATE" in current_entities:
                effective_intent = "booking"
            # If the last user input had a product inquiry intent and current input has a PRODUCT/NORP entity, assume product_inquiry
            elif last_user_intent == "product_inquiry" and ("PRODUCT" in current_entities or "NORP" in current_entities):
                effective_intent = "product_inquiry"

    # Append current user input to history
    history.append({"role": "user", "text": user_input})

    response_text = random.choice(responses.get(effective_intent, responses["unknown"])) # Default response based on effective intent

    # Logic for booking intent
    if effective_intent == "booking":
        date_found = False
        date = None
        if "DATE" in current_entities:
            date = current_entities["DATE"][0]
            response_text = random.choice(responses["booking"]) + f" How about {date}?"
            date_found = True
        elif not date_found:
            # Check conversation history for a date if not found in current input
            # Exclude the very last entry (current user input) from history search for previous date
            for entry in reversed(history[:-1]):
                if entry["role"] == "user":
                    prev_entities = extract_entities(entry["text"])
                    if "DATE" in prev_entities:
                        date = prev_entities["DATE"][0]
                        response_text = random.choice(responses["booking"]) + f" Based on our previous chat, how about {date}?"
                        date_found = True
                        break
            if not date_found:
                response_text = random.choice(responses["booking"]) + " What date would you prefer?"

    # Logic for product inquiry intent
    elif effective_intent == "product_inquiry":
        product = None
        if "PRODUCT" in current_entities:
            product = current_entities["PRODUCT"][0]
        elif "NORP" in current_entities: # Sometimes product names might be tagged as NORP (Nationalities, Religious or Political Groups) or other entities if SpaCy doesn't know them.
            product = current_entities["NORP"][0]
        else: # Check history for product if not found in current or current_entities
             for entry in reversed(history[:-1]):
                if entry["role"] == "user":
                    prev_entities = extract_entities(entry["text"])
                    if "PRODUCT" in prev_entities:
                        product = prev_entities["PRODUCT"][0]
                        break
                    elif "NORP" in prev_entities: # Also check for NORP in history for product names
                        product = prev_entities["NORP"][0]
                        break

        if product:
             response_text = random.choice(responses["product_inquiry"]) + f" Can you tell me more about the {product}?"
        else:
             response_text = random.choice(responses["product_inquiry"])

    # Append chatbot response to history
    history.append({"role": "chatbot", "text": response_text})

    return response_text

# Test the enhanced chatbot with conversation history
conversation_history = []

print("User: Hello, I'd like to book an appointment.")
resp = chatbot_response("Hello, I'd like to book an appointment.", conversation_history)
print("Chatbot:", resp)

print("User: I want it for next Tuesday.")
resp = chatbot_response("I want it for next Tuesday.", conversation_history)
print("Chatbot:", resp)

print("User: I'm looking for an iPhone.")
resp = chatbot_response("I'm looking for an iPhone.", conversation_history)
print("Chatbot:", resp)

print("User: Goodbye.")
resp = chatbot_response("Goodbye.", conversation_history)
print("Chatbot:", resp)

print("User: What is the weather like?")
resp = chatbot_response("What is the weather like?", conversation_history)
print("Chatbot:", resp)

print("Full Conversation History:", conversation_history)

## Evaluate Chatbot Performance

### Subtask:
Introduce methods to evaluate the chatbot's responses. This could include using metrics like BLEU or ROUGE for quality of generation, or setting up a framework for human evaluation to assess relevance and naturalness. Note to provide metrics and evaluation results.


**Reasoning**:
The first step is to install the `nltk` library, which is required for calculating BLEU scores, a metric for evaluating text generation quality.



In [ ]:
get_ipython().system('pip install nltk')
import nltk
nltk.download('punkt')

print("NLTK and punkt tokenizer downloaded and ready for use.")

**Reasoning**:
The next step is to create test cases, define an evaluation function using BLEU score, and then apply it to assess the chatbot's performance, as per the instructions.



In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize

# 2. Create a list of test cases
test_cases = [
    {
        "user_input": "Hello, I'd like to book an appointment.",
        "reference_response": "When would you like to book? What date would you prefer?"
    },
    {
        "user_input": "I want it for next Tuesday.",
        "reference_response": "When would you like to book? How about next Tuesday?"
    },
    {
        "user_input": "I'm looking for an iPhone.",
        "reference_response": "Which product are you interested in?"
    },
    {
        "user_input": "Goodbye.",
        "reference_response": "Have a great day!"
    },
    {
        "user_input": "What is the weather like?",
        "reference_response": "I'm not sure I understand that."
    },
    {
        "user_input": "I need help with my laptop.",
        "reference_response": "How can I assist you with support? Please describe your issue."
    },
    {
        "user_input": "Can you book a meeting for tomorrow?",
        "reference_response": "When would you like to book? How about tomorrow?"
    }
]

# 3. Define an evaluation function
def evaluate_chatbot(test_case):
    user_input = test_case["user_input"]
    reference_response = test_case["reference_response"]

    # Use a fresh history for each test case to avoid cross-contamination
    current_conversation_history = []
    chatbot_actual_response = chatbot_response(user_input, current_conversation_history)

    # Tokenize responses for BLEU score calculation
    # The reference is a list of lists of tokens (or single list if only one reference)
    # The candidate is a single list of tokens
    reference_tokens = [word_tokenize(reference_response.lower())]
    candidate_tokens = word_tokenize(chatbot_actual_response.lower())

    # Calculate BLEU score
    # Weights are for 1-gram, 2-gram, 3-gram, 4-gram. Adjust as needed.
    # For short sentences, using higher order n-grams might result in 0.0 scores.
    # Using uniform weights for simplicity.
    score = sentence_bleu(reference_tokens, candidate_tokens, weights=(0.25, 0.25, 0.25, 0.25))

    return chatbot_actual_response, score

# 4. Iterate through the test cases and print BLEU scores
bleu_scores = []
print("--- Chatbot Evaluation Results ---")
for i, tc in enumerate(test_cases):
    actual_response, score = evaluate_chatbot(tc)
    bleu_scores.append(score)
    print(f"\nTest Case {i+1}:")
    print(f"  User Input: '{tc['user_input']}'")
    print(f"  Reference:  '{tc['reference_response']}'")
    print(f"  Chatbot:    '{actual_response}'")
    print(f"  BLEU Score: {score:.4f}")

# 5. Calculate and print the average BLEU score
if bleu_scores:
    average_bleu_score = sum(bleu_scores) / len(bleu_scores)
    print(f"\n--- Overall Performance ---")
    print(f"Average BLEU Score across {len(test_cases)} test cases: {average_bleu_score:.4f}")
else:
    print("No test cases to evaluate.")

### Chatbot Architectural Diagram


```mermaid
graph TD
    A[User] -->|Input Text| B(User Interface);
    B -->|User Input| C[Intent Recognizer];
    B -->|User Input| D[Entity Extractor];

    C -->|Identified Intent| E{Context Manager};
    D -->|Extracted Entities| E;

    E -->|Contextualized Info| F[Response Generator];

    subgraph External Resources
        G[Pre-trained NLP Models] --> C;
        H[SpaCy Model] --> D;
        I[Response Templates/Knowledge Base] --> F;
    end

    F -->|Chatbot Response| B;
    B -->|Display Response| A;

    style A fill:#f9f,stroke:#333,stroke-width:2px;
    style B fill:#bbf,stroke:#333,stroke-width:2px;
    style C fill:#ccf,stroke:#333,stroke-width:2px;
    style D fill:#cfc,stroke:#333,stroke-width:2px;
    style E fill:#ffc,stroke:#333,stroke-width:2px;
    style F fill:#fcf,stroke:#333,stroke-width:2px;
    style G fill:#eee,stroke:#333,stroke-width:2px;
    style H fill:#eee,stroke:#333,stroke-width:2px;
    style I fill:#eee,stroke:#333,stroke-width:2px;
```

**Reasoning**:
The previous execution failed because NLTK's `word_tokenize` implicitly requires the `punkt_tab` resource, which was not downloaded. The error message explicitly suggests downloading `punkt_tab`. I will modify the previous `nltk.download` command to include `punkt_tab`.



In [ ]:
get_ipython().system('pip install nltk')
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

print("NLTK and punkt tokenizer downloaded and ready for use.")

**Reasoning**:
With `punkt_tab` now downloaded, the previous code to evaluate the chatbot using BLEU scores should now execute without a `LookupError`. I will re-run the evaluation code to generate and display the performance metrics.



In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize

# 2. Create a list of test cases
test_cases = [
    {
        "user_input": "Hello, I'd like to book an appointment.",
        "reference_response": "When would you like to book? What date would you prefer?"
    },
    {
        "user_input": "I want it for next Tuesday.",
        "reference_response": "When would you like to book? How about next Tuesday?"
    },
    {
        "user_input": "I'm looking for an iPhone.",
        "reference_response": "Which product are you interested in?"
    },
    {
        "user_input": "Goodbye.",
        "reference_response": "Have a great day!"
    },
    {
        "user_input": "What is the weather like?",
        "reference_response": "I'm not sure I understand that."
    },
    {
        "user_input": "I need help with my laptop.",
        "reference_response": "How can I assist you with support? Please describe your issue."
    },
    {
        "user_input": "Can you book a meeting for tomorrow?",
        "reference_response": "When would you like to book? How about tomorrow?"
    }
]

# 3. Define an evaluation function
def evaluate_chatbot(test_case):
    user_input = test_case["user_input"]
    reference_response = test_case["reference_response"]

    # Use a fresh history for each test case to avoid cross-contamination
    current_conversation_history = []
    chatbot_actual_response = chatbot_response(user_input, current_conversation_history)

    # Tokenize responses for BLEU score calculation
    # The reference is a list of lists of tokens (or single list if only one reference)
    # The candidate is a single list of tokens
    reference_tokens = [word_tokenize(reference_response.lower())]
    candidate_tokens = word_tokenize(chatbot_actual_response.lower())

    # Calculate BLEU score
    # Weights are for 1-gram, 2-gram, 3-gram, 4-gram. Adjust as needed.
    # For short sentences, using higher order n-grams might result in 0.0 scores.
    # Using uniform weights for simplicity.
    score = sentence_bleu(reference_tokens, candidate_tokens, weights=(0.25, 0.25, 0.25, 0.25))

    return chatbot_actual_response, score

# 4. Iterate through the test cases and print BLEU scores
bleu_scores = []
print("--- Chatbot Evaluation Results ---")
for i, tc in enumerate(test_cases):
    actual_response, score = evaluate_chatbot(tc)
    bleu_scores.append(score)
    print(f"\nTest Case {i+1}:")
    print(f"  User Input: '{tc['user_input']}'")
    print(f"  Reference:  '{tc['reference_response']}'")
    print(f"  Chatbot:    '{actual_response}'")
    print(f"  BLEU Score: {score:.4f}")

# 5. Calculate and print the average BLEU score
if bleu_scores:
    average_bleu_score = sum(bleu_scores) / len(bleu_scores)
    print(f"\n--- Overall Performance ---")
    print(f"Average BLEU Score across {len(test_cases)} test cases: {average_bleu_score:.4f}")
else:
    print("No test cases to evaluate.")

**Reasoning**:
The previous evaluation showed low BLEU scores for some test cases due to incorrect intent prioritization and lack of conversation history during evaluation. I will reorder the intent checks in `recognize_intent` to prioritize 'support' correctly and modify the evaluation loop to pass a persistent `conversation_history` to `chatbot_response` to enable proper context management across test cases. After these fixes, I will re-run the evaluation.



In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize

# Existing responses dictionary (unchanged)
responses = {
    "greeting": ["Hello!", "Hi there!", "Welcome!"],
    "booking": ["Sure! What date would you prefer?", "When would you like to book?"],
    "farewell": ["Goodbye!", "Have a great day!"],
    "product_inquiry": ["Which product are you interested in?", "Tell me more about the product you're looking for."],
    "support": ["How can I assist you with support?", "Please describe your issue."],
    "unknown": ["I'm not sure I understand that."]
}

# Modified recognize_intent to prioritize 'support' over 'product_inquiry'
def recognize_intent(user_input):
    user_input_lower = user_input.lower()
    # Prioritize more specific intents first
    if "book" in user_input_lower or "appointment" in user_input_lower:
        return "booking"
    elif "support" in user_input_lower or "help" in user_input_lower or "issue" in user_input_lower:
        return "support"
    elif "product" in user_input_lower or "item" in user_input_lower or "iphone" in user_input_lower or "laptop" in user_input_lower:
        return "product_inquiry"
    elif "hello" in user_input_lower or "hi" in user_input_lower:
        return "greeting"
    elif "bye" in user_input_lower or "goodbye" in user_input_lower:
        return "farewell"
    else:
        return "unknown"

# Existing extract_entities function (unchanged)
import spacy
nlp = spacy.load("en_core_web_sm")

def extract_entities(user_input):
    doc = nlp(user_input)
    entities = {}
    for ent in doc.ents:
        if ent.label_ not in entities:
            entities[ent.label_] = []
        entities[ent.label_].append(ent.text)
    return entities

# Existing chatbot_response function (unchanged, as its logic is correct for context)
def chatbot_response(user_input, history):
    current_intent = recognize_intent(user_input)
    current_entities = extract_entities(user_input)

    # Determine effective intent, considering history
    effective_intent = current_intent
    if current_intent == "unknown" and history:
        last_user_entry = None
        for entry in reversed(history): # Iterate backwards to find the last user input
            if entry["role"] == "user":
                last_user_entry = entry
                break
        if last_user_entry:
            last_user_intent = recognize_intent(last_user_entry["text"])
            # If the last user input had a booking intent and current input has a DATE entity, assume booking
            if last_user_intent == "booking" and "DATE" in current_entities:
                effective_intent = "booking"
            # If the last user input had a product inquiry intent and current input has a PRODUCT/NORP entity, assume product_inquiry
            elif last_user_intent == "product_inquiry" and ("PRODUCT" in current_entities or "NORP" in current_entities):
                effective_intent = "product_inquiry"

    # Append current user input to history
    history.append({"role": "user", "text": user_input})

    response_text = random.choice(responses.get(effective_intent, responses["unknown"])) # Default response based on effective intent

    # Logic for booking intent
    if effective_intent == "booking":
        date_found = False
        date = None
        if "DATE" in current_entities:
            date = current_entities["DATE"][0]
            response_text = random.choice(responses["booking"]) + f" How about {date}?"
            date_found = True
        elif not date_found:
            # Check conversation history for a date if not found in current or current_entities
            # Exclude the very last entry (current user input) from history search for previous date
            for entry in reversed(history[:-1]):
                if entry["role"] == "user":
                    prev_entities = extract_entities(entry["text"])
                    if "DATE" in prev_entities:
                        date = prev_entities["DATE"][0]
                        response_text = random.choice(responses["booking"]) + f" Based on our previous chat, how about {date}?"
                        date_found = True
                        break
            if not date_found:
                response_text = random.choice(responses["booking"]) + " What date would you prefer?"

    # Logic for product inquiry intent
    elif effective_intent == "product_inquiry":
        product = None
        if "PRODUCT" in current_entities:
            product = current_entities["PRODUCT"][0]
        elif "NORP" in current_entities: # Sometimes product names might be tagged as NORP (Nationalities, Religious or Political Groups) or other entities if SpaCy doesn't know them.
            product = current_entities["NORP"][0]
        else: # Check history for product if not found in current or current_entities
             for entry in reversed(history[:-1]):
                if entry["role"] == "user":
                    prev_entities = extract_entities(entry["text"])
                    if "PRODUCT" in prev_entities:
                        product = prev_entities["PRODUCT"][0]
                        break
                    elif "NORP" in prev_entities: # Also check for NORP in history for product names
                        product = prev_entities["NORP"][0]
                        break

        if product:
             response_text = random.choice(responses["product_inquiry"]) + f" Can you tell me more about the {product}?"
        else:
             response_text = random.choice(responses["product_inquiry"])
    elif effective_intent == "support":
        response_text = random.choice(responses["support"])

    # Append chatbot response to history
    history.append({"role": "chatbot", "text": response_text})

    return response_text

# 2. Create a list of test cases
test_cases = [
    {
        "user_input": "Hello, I'd like to book an appointment.",
        "reference_response": "When would you like to book? What date would you prefer?"
    },
    {
        "user_input": "I want it for next Tuesday.",
        "reference_response": "When would you like to book? How about next Tuesday?"
    },
    {
        "user_input": "I'm looking for an iPhone.",
        "reference_response": "Which product are you interested in?"
    },
    {
        "user_input": "Goodbye.",
        "reference_response": "Have a great day!"
    },
    {
        "user_input": "What is the weather like?",
        "reference_response": "I'm not sure I understand that."
    },
    {
        "user_input": "I need help with my laptop.",
        "reference_response": "How can I assist you with support? Please describe your issue."
    },
    {
        "user_input": "Can you book a meeting for tomorrow?",
        "reference_response": "When would you like to book? How about tomorrow?"
    }
]

# 3. Define an evaluation function that maintains history
def evaluate_chatbot_with_history(test_case, history):
    user_input = test_case["user_input"]
    reference_response = test_case["reference_response"]

    chatbot_actual_response = chatbot_response(user_input, history) # Pass history to chatbot_response

    # Tokenize responses for BLEU score calculation
    reference_tokens = [word_tokenize(reference_response.lower())]
    candidate_tokens = word_tokenize(chatbot_actual_response.lower())

    # Calculate BLEU score
    # Using uniform weights for simplicity.
    score = sentence_bleu(reference_tokens, candidate_tokens, weights=(0.25, 0.25, 0.25, 0.25))

    return chatbot_actual_response, score

# 4. Iterate through the test cases and print BLEU scores
bleu_scores = []
# Initialize conversation history for evaluation, persistent across test cases
evaluation_conversation_history = []
print("--- Chatbot Evaluation Results ---")
for i, tc in enumerate(test_cases):
    actual_response, score = evaluate_chatbot_with_history(tc, evaluation_conversation_history) # Call evaluation function with history
    bleu_scores.append(score)
    print(f"\nTest Case {i+1}:")
    print(f"  User Input: '{tc['user_input']}'")
    print(f"  Reference:  '{tc['reference_response']}'")
    print(f"  Chatbot:    '{actual_response}'")
    print(f"  BLEU Score: {score:.4f}")

# 5. Calculate and print the average BLEU score
if bleu_scores:
    average_bleu_score = sum(bleu_scores) / len(bleu_scores)
    print(f"\n--- Overall Performance ---")
    print(f"Average BLEU Score across {len(test_cases)} test cases: {average_bleu_score:.4f}")
else:
    print("No test cases to evaluate.")

**Reasoning**:
To fully address the subtask of evaluating chatbot performance, I will introduce another common metric: ROUGE scores. This involves installing the `rouge_score` library and modifying the `evaluate_chatbot_with_history` function to calculate and display both BLEU and ROUGE scores for each test case, along with their averages.



In [ ]:
get_ipython().system('pip install rouge_score')
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
from rouge_score import rouge_scorer
import random
import spacy

# Existing responses dictionary (unchanged)
responses = {
    "greeting": ["Hello!", "Hi there!", "Welcome!"],
    "booking": ["Sure! What date would you prefer?", "When would you like to book?"],
    "farewell": ["Goodbye!", "Have a great day!"],
    "product_inquiry": ["Which product are you interested in?", "Tell me more about the product you're looking for."],
    "support": ["How can I assist you with support?", "Please describe your issue."],
    "unknown": ["I'm not sure I understand that."]
}

# Modified recognize_intent to prioritize 'support' over 'product_inquiry'
def recognize_intent(user_input):
    user_input_lower = user_input.lower()
    # Prioritize more specific intents first
    if "book" in user_input_lower or "appointment" in user_input_lower:
        return "booking"
    elif "support" in user_input_lower or "help" in user_input_lower or "issue" in user_input_lower:
        return "support"
    elif "product" in user_input_lower or "item" in user_input_lower or "iphone" in user_input_lower or "laptop" in user_input_lower:
        return "product_inquiry"
    elif "hello" in user_input_lower or "hi" in user_input_lower:
        return "greeting"
    elif "bye" in user_input_lower or "goodbye" in user_input_lower:
        return "farewell"
    else:
        return "unknown"

# Existing extract_entities function (unchanged)
nlp = spacy.load("en_core_web_sm")

def extract_entities(user_input):
    doc = nlp(user_input)
    entities = {}
    for ent in doc.ents:
        if ent.label_ not in entities:
            entities[ent.label_] = []
        entities[ent.label_].append(ent.text)
    return entities

# Existing chatbot_response function (unchanged, as its logic is correct for context)
def chatbot_response(user_input, history):
    current_intent = recognize_intent(user_input)
    current_entities = extract_entities(user_input)

    # Determine effective intent, considering history
    effective_intent = current_intent
    if current_intent == "unknown" and history:
        last_user_entry = None
        for entry in reversed(history): # Iterate backwards to find the last user input
            if entry["role"] == "user":
                last_user_entry = entry
                break
        if last_user_entry:
            last_user_intent = recognize_intent(last_user_entry["text"])
            # If the last user input had a booking intent and current input has a DATE entity, assume booking
            if last_user_intent == "booking" and "DATE" in current_entities:
                effective_intent = "booking"
            # If the last user input had a product inquiry intent and current input has a PRODUCT/NORP entity, assume product_inquiry
            elif last_user_intent == "product_inquiry" and ("PRODUCT" in current_entities or "NORP" in current_entities):
                effective_intent = "product_inquiry"

    # Append current user input to history
    history.append({"role": "user", "text": user_input})

    response_text = random.choice(responses.get(effective_intent, responses["unknown"])) # Default response based on effective intent

    # Logic for booking intent
    if effective_intent == "booking":
        date_found = False
        date = None
        if "DATE" in current_entities:
            date = current_entities["DATE"][0]
            response_text = random.choice(responses["booking"]) + f" How about {date}?"
            date_found = True
        elif not date_found:
            # Check conversation history for a date if not found in current or current_entities
            # Exclude the very last entry (current user input) from history search for previous date
            for entry in reversed(history[:-1]):
                if entry["role"] == "user":
                    prev_entities = extract_entities(entry["text"])
                    if "DATE" in prev_entities:
                        date = prev_entities["DATE"][0]
                        response_text = random.choice(responses["booking"]) + f" Based on our previous chat, how about {date}?"
                        date_found = True
                        break
            if not date_found:
                response_text = random.choice(responses["booking"]) + " What date would you prefer?"

    # Logic for product inquiry intent
    elif effective_intent == "product_inquiry":
        product = None
        if "PRODUCT" in current_entities:
            product = current_entities["PRODUCT"][0]
        elif "NORP" in current_entities: # Sometimes product names might be tagged as NORP (Nationalities, Religious or Political Groups) or other entities if SpaCy doesn't know them.
            product = current_entities["NORP"][0]
        else: # Check history for product if not found in current or current_entities
             for entry in reversed(history[:-1]):
                if entry["role"] == "user":
                    prev_entities = extract_entities(entry["text"])
                    if "PRODUCT" in prev_entities:
                        product = prev_entities["PRODUCT"][0]
                        break
                    elif "NORP" in prev_entities: # Also check for NORP in history for product names
                        product = prev_entities["NORP"][0]
                        break

        if product:
             response_text = random.choice(responses["product_inquiry"]) + f" Can you tell me more about the {product}?"
        else:
             response_text = random.choice(responses["product_inquiry"])
    elif effective_intent == "support":
        response_text = random.choice(responses["support"])

    # Append chatbot response to history
    history.append({"role": "chatbot", "text": response_text})

    return response_text

# 2. Create a list of test cases
test_cases = [
    {
        "user_input": "Hello, I'd like to book an appointment.",
        "reference_response": "When would you like to book? What date would you prefer?"
    },
    {
        "user_input": "I want it for next Tuesday.",
        "reference_response": "When would you like to book? How about next Tuesday?"
    },
    {
        "user_input": "I'm looking for an iPhone.",
        "reference_response": "Which product are you interested in?"
    },
    {
        "user_input": "Goodbye.",
        "reference_response": "Have a great day!"
    },
    {
        "user_input": "What is the weather like?",
        "reference_response": "I'm not sure I understand that."
    },
    {
        "user_input": "I need help with my laptop.",
        "reference_response": "How can I assist you with support? Please describe your issue."
    },
    {
        "user_input": "Can you book a meeting for tomorrow?",
        "reference_response": "When would you like to book? How about tomorrow?"
    }
]

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# 3. Define an evaluation function that maintains history and calculates ROUGE
def evaluate_chatbot_with_history_and_rouge(test_case, history):
    user_input = test_case["user_input"]
    reference_response = test_case["reference_response"]

    chatbot_actual_response = chatbot_response(user_input, history) # Pass history to chatbot_response

    # Calculate BLEU score
    reference_tokens = [word_tokenize(reference_response.lower())]
    candidate_tokens = word_tokenize(chatbot_actual_response.lower())
    bleu_score = sentence_bleu(reference_tokens, candidate_tokens, weights=(0.25, 0.25, 0.25, 0.25))

    # Calculate ROUGE scores
    rouge_scores = scorer.score(reference_response, chatbot_actual_response)

    return chatbot_actual_response, bleu_score, rouge_scores

# 4. Iterate through the test cases and print BLEU and ROUGE scores
bleu_scores = []
rouge1_fmeasures = []
rouge2_fmeasures = []
rougeL_fmeasures = []

# Initialize conversation history for evaluation, persistent across test cases
evaluation_conversation_history = []
print("--- Chatbot Evaluation Results ---")
for i, tc in enumerate(test_cases):
    actual_response, bleu_score, rouge_scores = evaluate_chatbot_with_history_and_rouge(tc, evaluation_conversation_history) # Call evaluation function with history
    bleu_scores.append(bleu_score)
    rouge1_fmeasures.append(rouge_scores['rouge1'].fmeasure)
    rouge2_fmeasures.append(rouge_scores['rouge2'].fmeasure)
    rougeL_fmeasures.append(rouge_scores['rougeL'].fmeasure)

    print(f"\nTest Case {i+1}:")
    print(f"  User Input: '{tc['user_input']}'")
    print(f"  Reference:  '{tc['reference_response']}'")
    print(f"  Chatbot:    '{actual_response}'")
    print(f"  BLEU Score: {bleu_score:.4f}")
    print(f"  ROUGE-1 F1: {rouge_scores['rouge1'].fmeasure:.4f}")
    print(f"  ROUGE-2 F1: {rouge_scores['rouge2'].fmeasure:.4f}")
    print(f"  ROUGE-L F1: {rouge_scores['rougeL'].fmeasure:.4f}")

# 5. Calculate and print the average BLEU and ROUGE scores
if bleu_scores:
    average_bleu_score = sum(bleu_scores) / len(bleu_scores)
    average_rouge1 = sum(rouge1_fmeasures) / len(rouge1_fmeasures)
    average_rouge2 = sum(rouge2_fmeasures) / len(rouge2_fmeasures)
    average_rougeL = sum(rougeL_fmeasures) / len(rougeL_fmeasures)

    print(f"\n--- Overall Performance ---")
    print(f"Average BLEU Score across {len(test_cases)} test cases: {average_bleu_score:.4f}")
    print(f"Average ROUGE-1 F1 Score: {average_rouge1:.4f}")
    print(f"Average ROUGE-2 F1 Score: {average_rouge2:.4f}")
    print(f"Average ROUGE-L F1 Score: {average_rougeL:.4f}")
else:
    print("No test cases to evaluate.")

In [ ]:
print("Chatbot CLI. Type 'quit', 'exit', or 'bye' to end the conversation.")

# Initialize an empty list named conversation_history to store the dialogue.
conversation_history = []

# Create an infinite loop for continuous interaction
while True:
    user_input = input("You: ")

    # Check for termination commands
    if user_input.lower() in ["quit", "exit", "bye"]:
        print("Chatbot: Goodbye!")
        break

    # Call the chatbot_response function with the user's input and the conversation_history list
    chatbot_output = chatbot_response(user_input, conversation_history)

    # Print the chatbot's response
    print(f"Chatbot: {chatbot_output}")

    # (Optional) Print the current state of the conversation_history
    # print("\n--- Conversation History ---")
    # for entry in conversation_history:
    #     print(f"{entry['role'].capitalize()}: {entry['text']}")
    # print("---------------------------\n")


## Create README, Project Name, and Description

### Subtask:
Generate a comprehensive README for GitHub, including a suitable project name and description.


**Reasoning**:
To generate the README as requested by the subtask, I will define a multi-line string variable named `project_readme` containing all the specified sections and information for the chatbot project. This will be done in a new Python code cell.



In [ ]:
project_readme = """# Conversational AI Chatbot\n\n## Project Description\nThis project implements a sophisticated conversational AI chatbot designed to understand user intent, extract key entities from their queries, and maintain conversation context for more coherent and natural interactions. It serves as a foundational example of building intelligent dialogue systems using modern NLP techniques.\n\n## Features\n*   **Intent Recognition**: Identifies the user's underlying goal (e.g., 'booking', 'product_inquiry', 'support') to provide relevant responses.\n*   **Entity Extraction**: Utilizes SpaCy to pinpoint and extract crucial information like dates, products, or specific issues from user inputs.\n*   **Conversation Context Management**: Remembers previous turns in the conversation to handle follow-up questions and incomplete requests intelligently.\n*   **Response Generation**: Generates appropriate and contextually relevant responses based on recognized intent and extracted entities.\n*   **Performance Evaluation**: Includes methods for evaluating chatbot performance using metrics like BLEU and ROUGE scores.\n*   **Interactive Command-Line Interface (CLI)**: A simple interface for real-time interaction with the chatbot.\n\n## Technologies and Libraries Used\n*   **Python**: The core programming language.\n*   **SpaCy**: For advanced Natural Language Processing, specifically for entity extraction (`en_core_web_sm` model).\n*   **NLTK**: Used for text tokenization, which is essential for calculating BLEU scores.\n*   **rouge_score**: For calculating ROUGE metrics to evaluate response quality.\n*   **random**: For selecting random responses from predefined lists.\n*   **transformers (Hugging Face)**: Although not directly used in the final rule-based/contextual chatbot, it was explored for advanced response generation demonstrating its capabilities.\n\n## Setup and Installation\nTo set up and run the chatbot, follow these steps:\n\n1.  **Clone the repository (or save the notebook)**:\n    ```bash\n    # If it were a repository\n    # git clone <repository_url>\n    # cd conversational-ai-chatbot\n    ```\n\n2.  **Install dependencies**:\n    Ensure you have Python 3.8+ installed. Then, install the required libraries:\n    ```bash\n    pip install spacy nltk rouge_score\n    python -m spacy download en_core_web_sm\n    ```\n\n3.  **Run the Chatbot CLI**:\n    Execute the Python script (or the relevant cells in the notebook) that contains the `chatbot_response` function and the CLI loop.\n    ```bash\n    # Assuming your chatbot code is in a file named chatbot_cli.py\n    # python chatbot_cli.py\n    ```\n    In this notebook, you would run the final code cell that starts the interactive loop.\n\n## How to Interact with the Chatbot\nOnce the chatbot CLI is running:\n\n*   Type your messages in the prompt and press Enter.\n*   The chatbot will respond based on its understanding of your intent and extracted entities.\n*   To end the conversation, type `quit`, `exit`, or `bye` and press Enter.\n\n### Example Interactions:\n```\nChatbot CLI. Type 'quit', 'exit', or 'bye' to end the conversation.\nYou: Hello, I'd like to book an appointment.\nChatbot: When would you like to book? What date would you prefer?\nYou: I want it for next Tuesday.\nChatbot: When would you like to book? How about next Tuesday?\nYou: I'm looking for an iPhone.\nChatbot: Which product are you interested in?\nYou: I need help with my laptop.\nChatbot: How can I assist you with support? Please describe your issue.\nYou: Goodbye.\nChatbot: Goodbye!\n```\n"""

# Conversational AI Chatbot

## Project Description
This project implements a sophisticated conversational AI chatbot designed to understand user intent, extract key entities from their queries, and maintain conversation context for more coherent and natural interactions. It serves as a foundational example of building intelligent dialogue systems using modern NLP techniques.

## Features
*   **Intent Recognition**: Identifies the user's underlying goal (e.g., 'booking', 'product_inquiry', 'support') to provide relevant responses.
*   **Entity Extraction**: Utilizes SpaCy to pinpoint and extract crucial information like dates, products, or specific issues from user inputs.
*   **Conversation Context Management**: Remembers previous turns in the conversation to handle follow-up questions and incomplete requests intelligently.
*   **Response Generation**: Generates appropriate and contextually relevant responses based on recognized intent and extracted entities.
*   **Performance Evaluation**: Includes methods for evaluating chatbot performance using metrics like BLEU and ROUGE scores.
*   **Interactive Command-Line Interface (CLI)**: A simple interface for real-time interaction with the chatbot.

## Technologies and Libraries Used
*   **Python**: The core programming language.
*   **SpaCy**: For advanced Natural Language Processing, specifically for entity extraction (`en_core_web_sm` model).
*   **NLTK**: Used for text tokenization, which is essential for calculating BLEU scores.
*   **rouge_score**: For calculating ROUGE metrics to evaluate response quality.
*   **random**: For selecting random responses from predefined lists.
*   **transformers (Hugging Face)**: Although not directly used in the final rule-based/contextual chatbot, it was explored for advanced response generation demonstrating its capabilities.

## Setup and Installation
To set up and run the chatbot, follow these steps:

1.  **Clone the repository (or save the notebook)**:
    ```bash
    # If it were a repository
    # git clone <repository_url>
    # cd conversational-ai-chatbot
    ```

2.  **Install dependencies**:
    Ensure you have Python 3.8+ installed. Then, install the required libraries:
    ```bash
    pip install spacy nltk rouge_score
    python -m spacy download en_core_web_sm
    ```

3.  **Run the Chatbot CLI**:
    Execute the Python script (or the relevant cells in the notebook) that contains the `chatbot_response` function and the CLI loop.
    ```bash
    # Assuming your chatbot code is in a file named chatbot_cli.py
    # python chatbot_cli.py
    ```
    In this notebook, you would run the final code cell that starts the interactive loop.

## How to Interact with the Chatbot
Once the chatbot CLI is running:

*   Type your messages in the prompt and press Enter.
*   The chatbot will respond based on its understanding of your intent and extracted entities.
*   To end the conversation, type `quit`, `exit`, or `bye` and press Enter.

### Example Interactions:
```
Chatbot CLI. Type 'quit', 'exit', or 'bye' to end the conversation.
You: Hello, I'd like to book an appointment.
Chatbot: When would you like to book? What date would you prefer?
You: I want it for next Tuesday.
Chatbot: When would you like to book? How about next Tuesday?
You: I'm looking for an iPhone.
Chatbot: Which product are you interested in?
You: I need help with my laptop.
Chatbot: How can I assist you with support? Please describe your issue.
You: Goodbye.
Chatbot: Goodbye!
```

# Conversational AI Chatbot

## Project Description
This project implements a sophisticated conversational AI chatbot designed to understand user intent, extract key entities from their queries, and maintain conversation context for more coherent and natural interactions. It serves as a foundational example of building intelligent dialogue systems using modern NLP techniques.

## Features
*   **Intent Recognition**: Identifies the user's underlying goal (e.g., 'booking', 'product_inquiry', 'support') to provide relevant responses.
*   **Entity Extraction**: Utilizes SpaCy to pinpoint and extract crucial information like dates, products, or specific issues from user inputs.
*   **Conversation Context Management**: Remembers previous turns in the conversation to handle follow-up questions and incomplete requests intelligently.
*   **Response Generation**: Generates appropriate and contextually relevant responses based on recognized intent and extracted entities.
*   **Performance Evaluation**: Includes methods for evaluating chatbot performance using metrics like BLEU and ROUGE scores.
*   **Interactive Command-Line Interface (CLI)**: A simple interface for real-time interaction with the chatbot.

## Technologies and Libraries Used
*   **Python**: The core programming language.
*   **SpaCy**: For advanced Natural Language Processing, specifically for entity extraction (`en_core_web_sm` model).
*   **NLTK**: Used for text tokenization, which is essential for calculating BLEU scores.
*   **rouge_score**: For calculating ROUGE metrics to evaluate response quality.
*   **random**: For selecting random responses from predefined lists.
*   **transformers (Hugging Face)**: Although not directly used in the final rule-based/contextual chatbot, it was explored for advanced response generation demonstrating its capabilities.

## Setup and Installation
To set up and run the chatbot, follow these steps:

1.  **Clone the repository (or save the notebook)**:
    ```bash
    # If it were a repository
    # git clone <repository_url>
    # cd conversational-ai-chatbot
    ```

2.  **Install dependencies**:
    Ensure you have Python 3.8+ installed. Then, install the required libraries:
    ```bash
    pip install spacy nltk rouge_score
    python -m spacy download en_core_web_sm
    ```

3.  **Run the Chatbot CLI**:
    Execute the Python script (or the relevant cells in the notebook) that contains the `chatbot_response` function and the CLI loop.
    ```bash
    # Assuming your chatbot code is in a file named chatbot_cli.py
    # python chatbot_cli.py
    ```
    In this notebook, you would run the final code cell that starts the interactive loop.

## How to Interact with the Chatbot
Once the chatbot CLI is running:

*   Type your messages in the prompt and press Enter.
*   The chatbot will respond based on its understanding of your intent and extracted entities.
*   To end the conversation, type `quit`, `exit`, or `bye` and press Enter.

### Example Interactions:
```
Chatbot CLI. Type 'quit', 'exit', or 'bye' to end the conversation.
You: Hello, I'd like to book an appointment.
Chatbot: When would you like to book? What date would you prefer?
You: I want it for next Tuesday.
Chatbot: When would you like to book? How about next Tuesday?
You: I'm looking for an iPhone.
Chatbot: Which product are you interested in?
You: I need help with my laptop.
Chatbot: How can I assist you with support? Please describe your issue.
You: Goodbye.
Chatbot: Goodbye!
```

## Create Project Documentation and Presentation Assets

### Subtask:
Outline key elements for a good work presentation for the chatbot project.


## Create Project Documentation and Presentation Assets

### Subtask:
Outline key elements for a good work presentation for the chatbot project.

#### Instructions
1. Create a multi-line string variable named `presentation_outline`.
2. Populate this variable with a structured outline for a presentation, covering key aspects of the chatbot project.
3. The outline should include sections like 'Introduction', 'Problem Statement', 'Solution Overview', 'Technical Deep Dive (Intent Recognition, Entity Extraction, Context Management)', 'Performance Evaluation (Metrics & Results)', 'Interactive Demo', 'Future Work', and 'Q&A'.

**Reasoning**:
The subtask instructs to create a multi-line string variable named `presentation_outline` and populate it with a structured outline for the chatbot project presentation. I will create a code cell to define this variable with the specified content.



In [ ]:
presentation_outline = """# Chatbot Project Presentation Outline\n\n## 1. Introduction\n*   Project Title and Presenter(s)\n*   Brief overview of the chatbot's purpose and capabilities\n\n## 2. Problem Statement\n*   What challenge does this chatbot address?\n*   Why is an intelligent conversational agent needed?\n\n## 3. Solution Overview\n*   High-level architecture of the chatbot system\n*   Key components: Intent Recognition, Entity Extraction, Context Management, Response Generation\n\n## 4. Technical Deep Dive\n### 4.1. Intent Recognition\n*   How user intent is identified (e.g., keyword matching, classifier)\n*   Examples of supported intents (booking, product inquiry, support)\n### 4.2. Entity Extraction\n*   Role of SpaCy in identifying entities (dates, products)\n*   Demonstration of entity extraction in action\n### 4.3. Context Management\n*   How conversation history is maintained\n*   Addressing follow-up questions and maintaining coherence\n\n## 5. Performance Evaluation\n*   Metrics used: BLEU, ROUGE scores\n*   Summary of evaluation results and their interpretation\n*   Challenges and limitations of current evaluation\n\n## 6. Interactive Demo\n*   Live demonstration of the chatbot's functionalities\n*   Showcasing intent recognition, entity extraction, and context awareness\n\n## 7. Future Work\n*   Potential enhancements: more advanced NLP models, broader domain coverage, voice integration\n*   Deployment strategies and scalability considerations\n\n## 8. Q&A\n*   Open floor for questions and discussion\n"""

## Design a Conceptual Architectural Diagram

### Subtask:
Design a conceptual architectural diagram illustrating the chatbot's architecture, including data flow and pipeline.


**Reasoning**:
To design a conceptual architectural diagram, I will create a multi-line string variable containing the diagram represented in Mermaid syntax, illustrating the chatbot's architecture and data flow.



In [ ]:
architectural_diagram = """```mermaid
graph TD
    A[User] -->|Input Text| B(User Interface);
    B -->|User Input| C[Intent Recognizer];
    B -->|User Input| D[Entity Extractor];

    C -->|Identified Intent| E{Context Manager};
    D -->|Extracted Entities| E;

    E -->|Contextualized Info| F[Response Generator];

    subgraph External Resources
        G[Pre-trained NLP Models] --&gt; C;
        H[SpaCy Model] --&gt; D;
        I[Response Templates/Knowledge Base] --&gt; F;
    end

    F -->|Chatbot Response| B;
    B -->|Display Response| A;

    style A fill:#f9f,stroke:#333,stroke-width:2px;
    style B fill:#bbf,stroke:#333,stroke-width:2px;
    style C fill:#ccf,stroke:#333,stroke-width:2px;
    style D fill:#cfc,stroke:#333,stroke-width:2px;
    style E fill:#ffc,stroke:#333,stroke-width:2px;
    style F fill:#fcf,stroke:#333,stroke-width:2px;
    style G fill:#eee,stroke:#333,stroke-width:2px;
    style H fill:#eee,stroke:#333,stroke-width:2px;
    style I fill:#eee,stroke:#333,stroke-width:2px;
```"""
print("Architectural diagram variable created.")

## Final Task

### Subtask:
Review all the added components and documentation, ensuring the project is well-rounded, professionally presented, and ready for sharing.


## Summary:

### Data Analysis Key Findings

*   **Intent Recognition and Entity Extraction:**
    *   Initial keyword-based intent recognition showed limitations, misclassifying some user inputs (e.g., "I need help with my product." as 'product\_inquiry' instead of 'support').
    *   SpaCy successfully extracted entities like "tomorrow" (DATE) and "July 20th" (DATE).
    *   The integration of keyword-based intent recognition and SpaCy's entity extraction in initial tests resulted in some general responses due to limitations in the intent classification.
*   **Conversation Context Management:**
    *   The chatbot successfully maintained conversation history across turns, storing both user inputs and chatbot responses.
    *   Refined logic in `recognize_intent` and `chatbot_response` improved context handling, allowing the chatbot to correctly interpret follow-up questions (e.g., interpreting "next Tuesday" as a date for a previously initiated booking).
*   **Chatbot Performance Evaluation:**
    *   The chatbot's performance was quantitatively assessed using BLEU and ROUGE metrics.
    *   After refinements, the average BLEU Score was 0.5258.
    *   The average ROUGE-1 F1 Score was 0.6314, ROUGE-2 F1 Score was 0.5659, and ROUGE-L F1 Score was 0.6314.
*   **Simple Interactive User Interface:**
    *   A functional command-line interface (CLI) was successfully developed, enabling continuous user interaction, displaying chatbot responses, and supporting termination commands.
*   **Project Documentation and Presentation Assets:**
    *   A comprehensive README file was generated, including project name ("Conversational AI Chatbot"), description, features, technologies, setup instructions, and example interactions.
    *   A structured presentation outline was created, covering key project aspects from introduction to future work and Q\&A.
    *   A conceptual architectural diagram was designed using Mermaid syntax, illustrating the chatbot's components and data flow.

### Insights or Next Steps

*   **Enhance Intent Recognition with Machine Learning:** Transition from a keyword-based intent recognition system to a machine learning-based approach (e.g., using a small trained classifier with libraries like scikit-learn or even more advanced transformer models) to improve accuracy and reduce misclassifications for more nuanced user queries.
*   **Integrate Human Evaluation for Response Quality:** While BLEU and ROUGE provide quantitative metrics, consider incorporating a framework for human evaluation to assess the naturalness, relevance, and overall helpfulness of the chatbot's responses, providing more qualitative insights for further improvements.
